In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import MaxPooling2D, Conv2D, Dense, Flatten
from keras import layers, models
import pandas as pd
import numpy as np

import librosa
import librosa.display
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
duration = 20
offset = 10

In [3]:
def load_csv():
    data_csv = pd.read_csv("data.csv", skipinitialspace=True)
    genre = data_csv[["track id", "genre"]]
    labels = data_csv.drop(
        columns=["mood", "age", "gender", "disliked", "liked", "mother tongue", "genre"]
    )  # unnecessary columns
    labels = labels.groupby("track id").mean()
    for column in labels.columns:
        labels[column] = labels[column].apply(lambda x: 1 if (x >= 0.5) else 0)
        print(labels[column].value_counts())
        print()
    genre_cleared = genre.drop_duplicates().set_index("track id")
    y = labels
    y["genre"] = genre_cleared["genre"]
    y = pd.get_dummies(y)
    return y


y = load_csv()

0    397
1      3
Name: amazement, dtype: int64

0    381
1     19
Name: solemnity, dtype: int64

0    373
1     27
Name: tenderness, dtype: int64

0    354
1     46
Name: nostalgia, dtype: int64

0    311
1     89
Name: calmness, dtype: int64

0    366
1     34
Name: power, dtype: int64

0    317
1     83
Name: joyful_activation, dtype: int64

0    355
1     45
Name: tension, dtype: int64

0    379
1     21
Name: sadness, dtype: int64



In [4]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desirex width
    :return: padded array
    """

    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2, 0)
    aa = max(0, xx - a - h)
    b = max(0, (yy - w) // 2)
    bb = max(yy - b - w, 0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode="constant")

In [5]:
def get_spec(y):
    spec = np.abs(librosa.stft(y, n_fft=256, hop_length=512))
    return spec

In [44]:
def load_audio():
    x_list = []
    audio_path = Path('audio')
    genres = audio_path.glob("*")
    for genre in genres:
        for i in range(1,101,1):
            offset = 10
            duration = 20
            print(f"{genre}/{i}.mp3")
            real_duration = int(librosa.get_duration(filename=f"{genre}/{i}.mp3"))
            if duration + offset >= real_duration:
               offset = 0
               duration = real_duration
        
            y, sr = librosa.load((f"{genre}/{i}.mp3"), duration=duration, offset=offset)
            #y, sr = librosa.load((f"{genre}/{i}.mp3"), duration=duration)    
            x = get_spec(y)
            x = padding(x, 128, 1000)
            x_list.append(np.expand_dims(x, 2))
    return x_list

x = load_audio()

audio\classical/1.mp3
audio\classical/2.mp3
audio\classical/3.mp3
audio\classical/4.mp3
audio\classical/5.mp3
audio\classical/6.mp3
audio\classical/7.mp3
audio\classical/8.mp3
audio\classical/9.mp3
audio\classical/10.mp3
audio\classical/11.mp3
audio\classical/12.mp3
audio\classical/13.mp3
audio\classical/14.mp3
audio\classical/15.mp3
audio\classical/16.mp3
audio\classical/17.mp3
audio\classical/18.mp3
audio\classical/19.mp3
audio\classical/20.mp3
audio\classical/21.mp3
audio\classical/22.mp3
audio\classical/23.mp3
audio\classical/24.mp3
audio\classical/25.mp3
audio\classical/26.mp3
audio\classical/27.mp3
audio\classical/28.mp3
audio\classical/29.mp3
audio\classical/30.mp3
audio\classical/31.mp3
audio\classical/32.mp3
audio\classical/33.mp3
audio\classical/34.mp3
audio\classical/35.mp3
audio\classical/36.mp3
audio\classical/37.mp3
audio\classical/38.mp3
audio\classical/39.mp3
audio\classical/40.mp3
audio\classical/41.mp3
audio\classical/42.mp3
audio\classical/43.mp3
audio\classical/44.m

In [45]:
input_shape = (128, 1000, 1)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation="relu"))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation="relu"))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(64, activation="relu"))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Dense(32, activation="relu"))
CNNmodel.add(layers.Dense(13, activation="sigmoid"))

In [46]:
CNNmodel.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=["accuracy"],
)

In [47]:
xd = np.asarray(x)

C:\Users\barto\AppData\Local\Temp\ipykernel_21276\3260485056.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xd = np.asarray(x)


ValueError: could not broadcast input array from shape (128,1000,1) into shape (128,)

In [48]:
xd.shape

(400, 128, 1000, 1)

In [49]:
y.shape

(400, 13)

In [50]:
history = CNNmodel.fit(
    x=xd,
    y=y,
    epochs=100,
    batch_size=1
)

Epoch 1/100
400/400 [==============================] - 4s 8ms/step - loss: 0.4346 - accuracy: 0.0750
Epoch 2/100
400/400 [==============================] - 3s 7ms/step - loss: 0.3858 - accuracy: 0.1250
Epoch 3/100
400/400 [==============================] - 3s 7ms/step - loss: 0.3548 - accuracy: 0.1550
Epoch 4/100
400/400 [==============================] - 3s 7ms/step - loss: 0.3161 - accuracy: 0.2525
Epoch 5/100
400/400 [==============================] - 3s 7ms/step - loss: 0.2752 - accuracy: 0.3175
Epoch 6/100
400/400 [==============================] - 3s 7ms/step - loss: 0.2313 - accuracy: 0.3375
Epoch 7/100
400/400 [==============================] - 3s 7ms/step - loss: 0.2119 - accuracy: 0.3275
Epoch 8/100
400/400 [==============================] - 3s 7ms/step - loss: 0.1680 - accuracy: 0.3800
Epoch 9/100
400/400 [==============================] - 3s 7ms/step - loss: 0.1597 - accuracy: 0.3675
Epoch 10/100
400/400 [==============================] - 3s 7ms/step - loss: 0.1516 - accura

In [58]:
g = CNNmodel.predict(xd)

13/13 [==============================] - 0s 20ms/step
